# Vis 1
## Data Processing

In [1]:
import altair as alt
from altair import datum
from vega_datasets import data
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

mapDataWithCountryNames= alt.topo_feature("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json",'name')
mapData = alt.topo_feature("https://vega.github.io/vega/data/world-110m.json",'countries')

locs = pd.read_csv('./pleiades-locations-latest.csv')
places = pd.read_csv('./pleiades-places-latest.csv')
names = pd.read_csv('./pleiades-names-latest.csv')

capitals = pd.read_json('http://techslides.com/demos/country-capitals.json')



print("locs.shape = ",locs.shape)
print("places.shape = ",places.shape)
print("names.shape = ",names.shape)
print("capitals.shape = ",capitals.shape)

locs.shape =  (40463, 25)
places.shape =  (37737, 26)
names.shape =  (34133, 26)
capitals.shape =  (245, 6)


In [2]:
# Data Prep

locsTen= locs[1:40468:10]   #takes ten'th item from locs
# print("locsTen.shape = ", locsTen.shape)

bathFilter=((locs.featureType==("bath,"))|(locs.featureType==("bath")))
bathData = locs[bathFilter]
bathPoints = bathData[1:bathData.size:10]
print("bathData.shape = ",bathData.shape)
print("bathPoints.shape = ",bathPoints.shape,"\n")



bathData.shape =  (75, 25)
bathPoints.shape =  (8, 25) 



In [3]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Visulisation

In [5]:
#prerequisites
colorRange = ['red','green','blue','yellow','orange','black','grey','brown','teal','purple','navy','cream']
click = alt.selection_multi(encodings=['color'])
centerVal = [20,45]
scaleVal = 700

# world map
background = alt.Chart(mapData).mark_geoshape(
    fill = '#E0E0E0',
    stroke = '#D0D0D0').project(
    center = centerVal,
    scale=scaleVal,
).properties(
    width=800,
    height=500
)


# points on background
settlementP = alt.Chart(bathData).mark_point(
    size=20
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='reprLong',
    latitude='reprLat',
    color=alt.Color('timePeriods',scale=alt.Scale(range=colorRange)),
    tooltip=['featureType','reprLatLong','maxDate','timePeriods']
).transform_filter(
    datum.timePeriods!="null"
)


# capital text
capitalsText = alt.Chart(capitals).mark_text(
    size=7, 
    font='calibri'
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='CapitalLongitude:Q',
    latitude='CapitalLatitude:Q',
    text = 'CapitalName'
)

mainMap = background + settlementP + capitalsText
mainMap

alt.LayerChart(...)